<a href="https://colab.research.google.com/github/irfnt/Adaboost-DSNN/blob/main/vit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
pip install -U tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.7 MB/s 


In [ ]:
import glob
import numpy as np
import time
import os
import tensorflow as tf
#import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
import pandas as pd
import datetime
from tensorflow.keras.utils import to_categorical

#save_path = './train_dense_net/ckpt/'


image_size = 512
learning_rate = 0.00001
weight_decay = 0.0001
input_shape = (512, 512, 1)
patch_size = 16
num_classes=8




BATCH_SIZE = 32
model_iter = 10
num_epochs = 200

save_path = F"/content/drive/MyDrive/VIT_512_CKPT/CKPT"


#################### Data Loading Pipeline  ############################
mos_df   = pd.read_csv('/content/drive/MyDrive/total image and file /train_csv_mult_file_smote.csv')
mos_np = np.array(mos_df)

images_data = '/content/drive/MyDrive/total image and file /image_512_pixel/'
x_images = []
yy_labels = []

for name_id in mos_np:
    image_name = name_id[0]
    image_label = name_id[2]                              # convert to numerical value (0,1,2,3,4,5,6,7)
    x_img = Image.open(images_data + image_name)
    #print(name_id[1])
    x_img_np = np.array(x_img)
    x_images.append(x_img_np)
    yy_labels.append(image_label)

x_images = np.array(x_images).astype('float32')
x_images = np.reshape(x_images,(-1,512,512,1))
x_images = x_images/255.0



yy_labels = np.array(yy_labels)                         # numeric labels
yy_labels_one_hot = to_categorical(yy_labels)           # converting numeric labels to one_hot_encoder
print(yy_labels_one_hot)
# yy_labels_one_hot = yy_labels


train_dataset = tf.data.Dataset.from_tensor_slices((x_images,yy_labels_one_hot))
train_dataset = train_dataset.batch(BATCH_SIZE)


# =============================================================================
# Model
# =============================================================================

num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
print(num_heads)
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024] # Size of the dense layers of the final classifier


data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
print((data_augmentation))


# # # Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(x_images)


def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.relu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x




class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size
    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches







# plt.figure(figsize=(4, 4))
# image = X_train[np.random.choice(range(X_train.shape[0]))]
# plt.imshow(image.astype("uint8"))
# plt.axis("off")

# resized_image = tf.image.resize(
#     tf.convert_to_tensor([image]), size=(image_size, image_size)
# )
# patches = Patches(patch_size)(resized_image)
# print(f"Image size: {image_size} X {image_size}")
# print(f"Patch size: {patch_size} X {patch_size}")
# print(f"Patches per image: {patches.shape[1]}")
# print(f"Elements per patch: {patches.shape[-1]}")


# n = int(np.sqrt(patches.shape[1]))
# plt.figure(figsize=(4, 4))
# for i, patch in enumerate(patches[0]):
#     ax = plt.subplot(n, n, i + 1)
#     patch_img = tf.reshape(patch, (patch_size, patch_size, 1))
#     plt.imshow(patch_img.numpy().astype("uint8"))
#     plt.axis("off")




class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded













def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=0.000001)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=0.000001)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])
  # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=0.000001)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)

    # Classify outputs.





    logits = layers.Dense(num_classes,activation='softmax')(features)


    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)

    return model




model = create_vit_classifier()

optimizer = tf.keras.optimizers.Adam(
    learning_rate=learning_rate
)
model.compile(
    optimizer=optimizer,
    loss=keras.losses.CategoricalCrossentropy(from_logits=True))




output_dir = F"/content/drive/MyDrive/VIT_512_CKPT/CKPT"
# =============================================================================
# iteration of model
# =============================================================================

#if epoch<=600:
  #continue

# if epoch<=30:
#   continue
for epoch in range(num_epochs):
  if epoch<=100:
    continue
    hist_lss = []

    for step, (input_image, target) in enumerate(train_dataset):


        #print(input_image.shape)
        hist = model.train_on_batch(x=input_image,y=target,reset_metrics=False, return_dict=True)
        hist_lss.append( hist['loss'])

    print('## Epoch  ======', epoch, 'loss_history = ',sum(hist_lss)/len(hist_lss))

    if epoch%model_iter == 0:
        print('epoch',epoch,"folder generation")
        os.mkdir(save_path+str(epoch))

        model.save(save_path+str(epoch))

data = pd.DataFrame(hist_lss)
data.to_csv(output_dir+'90.csv',header=True,index=False)

FileNotFoundError: ignored